In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Eczema/GSE150797'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression changes in atopic dermatitis after treatment with nb-UVB"
!Series_summary	"Background: Atopic dermatitis (AD) is a common inflammatory skin disease with broad impact on quality of life and on the health care system. The pathophysiology is not fully understood, but it is likely multifactorial involving immune dysfunction, altered skin barrier and environmental factors. Narrow band ultraviolet B (nb-UVB) treatment leads to normalization of the tissue and clinical improvement. However, knowledge of early changes in AD skin in response to nb-UVB is lacking and could provide important clues to decipher the disease mechanisms and potential new treatment targets."
!Series_summary	""
!Series_summary	"Objective: To map the early transcriptional changes in the skin in response to nb-UVB treatment."
!Series_summary	""
!Series_summary	"Results: When examining the early response after only three local UVB-treatments, gene expression analysis re

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

is_gene_available = True

# Defining data rows
trait_row = None  # Eczema data not explicitly specified
age_row = None    # Age data not explicitly specified
gender_row = 1    # Gender data found in key 1

# Define conversion functions
def convert_trait(value):
    # Since trait data is not available, this function is placeholder
    return None

def convert_age(value):
    return None  # Age data not available

def convert_gender(value):
    gender_mapping = {' Male': 1, ' Female': 0}
    key, val = value.split(':')
    return gender_mapping.get(val.strip(), None)

# Save cohort information
save_cohort_info('GSE150797', './preprocessed/Eczema/cohort_info.json', is_gene_available, trait_row is not None)
